# Pytorch Quickstart

## 数据处理

Pytorch 中对于数据的操作主要有两个接口：`torch.utils.data.DataLoader` 和 `torch.utils.data.Dataset`。`Dataset` 存储样本及其标签，`DataLoader` 在数据集 `Dataset` 上封装一个可迭代器。

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

Pytorch 提供了不同特定领域下的库，例如：TorchText, TorchVision 和 TorchAudio，每个领域都有其数据集。在该教学中，我们使用 TorchVision 数据集。

`torchvision.datasets` 模块包含许多真实世界视觉数据的数据集对象，如 CIFAR 和 [COCO](https://pytorch.org/vision/stable/datasets.html)。 在本教程中，我们使用的是 FashionMNIST 数据集。 每个 TorchVision 数据集都包含两个参数：`transform` 和 `target_transform`，分别用于修改样本和标签。

In [7]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26421880/26421880 [00:04<00:00, 5857540.15it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 189535.73it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3504517.93it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 20011378.12it/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



将`Dataset`作为一个参数传递给`DataLoader`，`DataLoader`就会在数据集上封装一个可迭代器，支持自动批处理、采样、洗牌和多进程数据加载。 在这里，我们将批量大小定义为 64，即数据加载器可迭代器中的每个元素都将返回一批 64 个特征和标签。

In [8]:
batch_size = 64

# 创建数据加载器
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X,y in test_dataloader:
    print(f"Shape of X[N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X[N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


了解更多有关在 PyTorch 中[加载数据的信息](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html)。

## 创建模型

要在 PyTorch 中定义一个神经网络，我们需要创建一个继承自 `nn.Module` 的类。 我们在 `__init__` 函数中定义网络的层，并在 `forward` 函数中指定数据如何通过网络。为了加速神经网络中的操作，我们将其移至 GPU 或 MPS（如果有的话）。

In [10]:
# 获取 CPU/GPU/MPS 设备以进行训练
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# 定义模型
class NeuralNetwork(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device=device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


了解更多有关在 PyTorch 中[构建神经网络的信息](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html)。

## 优化模型参数

要训练一个模型，我们需要一个[损失函数](https://pytorch.org/docs/stable/nn.html#loss-functions)和一个[优化器](https://pytorch.org/docs/stable/optim.html)。

In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

在单次训练循环中，模型对训练数据集（分批输入）进行预测，并反向传播预测误差以调整模型参数。

In [12]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 计算预测误差
        pred = model(X)
        loss = loss_fn(pred, y)

        # 反向传播
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}|{size:>5d}]")

我们还会根据测试数据集检查模型的性能，以确保它在学习。

In [18]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss:{test_loss:>8f} \n")

训练过程经过多次迭代（epochs）。 在每个迭代期间，模型都会学习参数，以便做出更好的预测。 我们打印模型在每个迭代的准确率和损失；我们希望看到准确率随着每个迭代的增加而增加，损失随着每个迭代的增加而减少。

In [19]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-----------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-----------------------------------
loss: 1.581332 [   64|60000]
loss: 1.547459 [ 6464|60000]
loss: 1.394014 [12864|60000]
loss: 1.471063 [19264|60000]
loss: 1.350625 [25664|60000]
loss: 1.333773 [32064|60000]
loss: 1.372505 [38464|60000]
loss: 1.281249 [44864|60000]
loss: 1.315042 [51264|60000]
loss: 1.220717 [57664|60000]
Test Error: 
 Accuracy: 63.6%, Avg loss:1.243624 

Epoch 2
-----------------------------------
loss: 1.317423 [   64|60000]
loss: 1.305370 [ 6464|60000]
loss: 1.136313 [12864|60000]
loss: 1.242699 [19264|60000]
loss: 1.116775 [25664|60000]
loss: 1.134712 [32064|60000]
loss: 1.177914 [38464|60000]
loss: 1.100940 [44864|60000]
loss: 1.134835 [51264|60000]
loss: 1.061547 [57664|60000]
Test Error: 
 Accuracy: 64.9%, Avg loss:1.078073 

Epoch 3
-----------------------------------
loss: 1.145181 [   64|60000]
loss: 1.156396 [ 6464|60000]
loss: 0.971395 [12864|60000]
loss: 1.104244 [19264|60000]
loss: 0.976730 [25664|60000]
loss: 1.004848 [32064|60000]
loss: 1.0616

了解更多有关[训练模型](https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html)的信息。

## 保存模型

保存模型的常用方法是序列化内部状态字典（包含模型参数）。

In [20]:
torch.save(model.state_dict(), "model.pth")
print("Saved Pytorch Model State to model.pth")

Saved Pytorch Model State to model.pth


## 加载模型

加载模型的过程包括重新创建模型结构和加载状态字典。

In [21]:
model = NeuralNetwork().to(device=device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


<All keys matched successfully>

现在可以利用这一模型进行预测。

In [22]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f"Predicted: {predicted}, Actual: {actual}")


Predicted: Ankle boot, Actual: Ankle boot


了解更多有关[保存和加载模型](https://pytorch.org/tutorials/beginner/basics/saveloadrun_tutorial.html)的信息。